## Práctica 1: Datos personales y anonimización

**Utilizando los datos recogidos en el fichero de datos abiertos de PDI de la UAM correspondiente al año 2022, realizar, para cada campo del fichero exceptuando aquellos cuyo nombre empieza por "cod_", "lat_" y "lon_", lo siguiente:**

* **Inferir el valor más probable de cada campo para el caso del profesor Ortigosa.**

* **Calcular el nivel de certeza (%) de que cada valor inferido corersponda realmente al profesor Ortigosa.**

* **Una breve descripción de infrencia utilizado en cada caso.**

#### Datos de partida y condiciones
Aseguramos que:
* El profesor Ortigosa estaba en 2022 en la UAM (aparece en el fichero).
* El profesor Ortigosa era de género masculino en 2022.
* El profesor Ortigosa pertenecía al Departamento de Ingeniería Informática en 2022.
* Puede utilizarse para el ejercicio cualquier otra información que se pueda obtener por cualquier vía adicional, pero debe especificarse claramente en la respuesta tanto la información usada como su origen.

In [1]:
!pip install pandas

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
csv_name = "uam-personal-pdi-2022-anonimizado.csv"

df = pd.read_csv(os.path.join(os.curdir, csv_name))

df = df.loc[:, ~df.columns.str.startswith(("cod_", "lat_", "lon_"))]

df

,IMPORTANTE,des_universidad,anio,des_pais_nacionalidad,des_continente_nacionalidad,des_agregacion_paises_nacionalidad,des_comunidad_residencia,des_provincia_residencia,des_municipio_residencia,des_genero,...,des_tipo_unidad_responsable,des_unidad_responsable,des_area_conocimiento,anio_incorporacion_ap,anio_incorpora_cuerpo_docente,num_trienios,num_quinquenios,num_sexenios,num_tesis,ind_investigador_principal
0,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Mujer,...,Departamento,"Departamento de Educación Física, Deporte y Mo...",Didáctica de la Expresión Corporal,NaN,NaN,7,4,1,NaN,N
1,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,FUENLABRADA,Mujer,...,Departamento,"Departamento de Educación Física, Deporte y Mo...",Didáctica de la Expresión Corporal,1996.0,2003.0,8,4,2,NaN,S
2,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Mujer,...,Departamento,"Departamento de Educación Física, Deporte y Mo...",Didáctica de la Expresión Corporal,NaN,NaN,12,5,0,NaN,N
3,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Mujer,...,Departamento,"Departamento de Educación Física, Deporte y Mo...",Didáctica de la Expresión Corporal,NaN,NaN,6,3,2,NaN,N
4,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Mujer,...,Departamento,"Departamento de Educación Física, Deporte y Mo...",Didáctica de la Expresión Corporal,1986.0,2000.0,12,4,3,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Mujer,...,Departamento,NaN,NaN,1984.0,1988.0,12,6,5,NaN,N
2635,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Islas Baleares,Illes Balears,PALMA,Hombre,...,No consta,NaN,NaN,1971.0,1985.0,12,0,0,NaN,N
2636,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,No consta,NaN,NaN,1987.0,2000.0,6,3,2,NaN,N
2637,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Mujer,...,No consta,NaN,NaN,NaN,NaN,0,0,0,NaN,N


In [4]:
# Primero tenemos que ver donde hay valores de NaN para tenerlos en cuenta de aquí en adelante
columnas_nan = [col for col in df.columns if df[col].isna().any()]
print(columnas_nan)

['des_comunidad_residencia', 'des_provincia_residencia', 'des_municipio_residencia', 'num_horas_semanales_tiempo_parcial', 'des_titulo_doctorado', 'des_pais_doctorado', 'des_continente_doctorado', 'des_agregacion_paises_doctorado', 'des_universidad_doctorado', 'anio_lectura_tesis', 'anio_expedicion_titulo_doctor', 'des_mencion_europea', 'des_unidad_responsable', 'des_area_conocimiento', 'anio_incorporacion_ap', 'anio_incorpora_cuerpo_docente', 'num_tesis']


In [5]:
# Utilizamos las variables pivote (que pertenecen al bloque 4 y 8) para realizar un filtrado inicial de los datos
# No hemos considerado los NaN para des_unidad_responsable ya que viene en el enunciado 

df_blq4 = df.loc[df['des_genero'] == "Hombre"]
df_blq8 = df_blq4[(df_blq4['des_unidad_responsable'] == "Departamento de Ingeniería Informática") & \
                  (df_blq4['des_area_conocimiento'] == "Lenguajes y Sistemas Informáticos") | \
                  (df_blq4['des_area_conocimiento'].isna())]
df_blq8

,IMPORTANTE,des_universidad,anio,des_pais_nacionalidad,des_continente_nacionalidad,des_agregacion_paises_nacionalidad,des_comunidad_residencia,des_provincia_residencia,des_municipio_residencia,des_genero,...,des_tipo_unidad_responsable,des_unidad_responsable,des_area_conocimiento,anio_incorporacion_ap,anio_incorpora_cuerpo_docente,num_trienios,num_quinquenios,num_sexenios,num_tesis,ind_investigador_principal
207,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,2003.0,2021.0,5,3,2,NaN,N
208,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,Italia,Europa,Europa meridional,Madrid,Madrid,ALCOBENDAS,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,0,0,0,NaN,N
209,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,1999.0,2005.0,7,5,4,NaN,S
213,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,0,0,0,NaN,N
216,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,2,1,0,NaN,N
218,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,1997.0,2001.0,8,6,5,NaN,N
219,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,"ROZAS DE MADRID, LAS",Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,2001.0,2021.0,5,2,3,NaN,N
220,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,5,1,0,NaN,N
226,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,1,0,0,NaN,N
230,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,TRES CANTOS,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,0,2,0,NaN,N


In [ ]:
# El bloque 1 (Universidad y año del conjunto de datos) no nos aporta información relevante.
# El bloque 2 (Nacionalidad) tampoco nos aporta información relevante.
# El bloque 3 (Residencia) tampoco nos aporta información relevante.
# El bloque 4 (Género) ya lo hemos utilizado.
# El bloque 5 (Contrato actual) no lo hemos utilizado.
# El bloque 6 (Cargo remunerado) no se puede utilizar.
# El bloque 7 (Doctorado):

df_blq7 = df_blq8[(df_blq8['des_titulo_doctorado'] == "Uno") & \
                  (df_blq8['des_pais_doctorado'] == "España") & \
                  (df_blq8['des_continente_doctorado'] == "Europa") & \
                  (df_blq8['des_agregacion_paises_doctorado'] == "Europa meridional") & \
                  (df_blq8['anio_lectura_tesis'] == 2000) | \
                  (df_blq8['des_titulo_doctorado'].isna()) | \
                  (df_blq8['des_pais_doctorado'].isna()) | \
                  (df_blq8['des_continente_doctorado'].isna()) | \
                  (df_blq8['des_agregacion_paises_doctorado'].isna()) | \
                  (df_blq8['anio_lectura_tesis'].isna())]

df_blq7

,IMPORTANTE,des_universidad,anio,des_pais_nacionalidad,des_continente_nacionalidad,des_agregacion_paises_nacionalidad,des_comunidad_residencia,des_provincia_residencia,des_municipio_residencia,des_genero,...,des_tipo_unidad_responsable,des_unidad_responsable,des_area_conocimiento,anio_incorporacion_ap,anio_incorpora_cuerpo_docente,num_trienios,num_quinquenios,num_sexenios,num_tesis,ind_investigador_principal
207,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,2003.0,2021.0,5,3,2,NaN,N
213,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,0,0,0,NaN,N
216,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,2,1,0,NaN,N
219,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,"ROZAS DE MADRID, LAS",Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,2001.0,2021.0,5,2,3,NaN,N
226,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,1,0,0,NaN,N
230,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,TRES CANTOS,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,0,2,0,NaN,N
234,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,Irán (República Islámica del),Asia,Asia meridional,Madrid,Madrid,COLMENAR VIEJO,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,9,2,0,NaN,N
236,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,"BOALO, EL",Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,5,1,1,NaN,N
241,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,8,2,0,NaN,N
243,LOS DATOS DE ESTA FILA NO CORRESPONDEN A UN SÓ...,Universidad Autónoma de Madrid,2022,Italia,Europa,Europa meridional,Madrid,Madrid,ALCOBENDAS,Hombre,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,0,0,0,NaN,N


In [ ]:
# El bloque 8 (Departamento) ya lo hemos utilizado.
# El bloque 9 (Cuerpo docente e investigador): 

df_blq9 = df_blq7[(df_blq7['anio_incorpora_cuerpo_docente'] == 2001) & \
                  (df_blq7['num_sexenios'] == 3) & \
                  (df_blq7['anio_incorpora_cuerpo_docente'].isna()) | \
                  (df_blq7['num_sexenios'].isna())]

df_blq9

# En algún lado la he liado porque no hay ninguno con esos requisitos 
# Pero tenemos información suficiente creo yo para encontrarlo con bastante bastante seguridad
# Sospecho que sea que se incorpora al cuerpo docente en otro año pero no sé seguro

,IMPORTANTE,des_universidad,anio,des_pais_nacionalidad,des_continente_nacionalidad,des_agregacion_paises_nacionalidad,des_comunidad_residencia,des_provincia_residencia,des_municipio_residencia,des_genero,...,des_tipo_unidad_responsable,des_unidad_responsable,des_area_conocimiento,anio_incorporacion_ap,anio_incorpora_cuerpo_docente,num_trienios,num_quinquenios,num_sexenios,num_tesis,ind_investigador_principal


In [ ]:
# El bloque 10 (Número de tesis) no se puede utilizar.
# El bloque 11 (Investigador principal) no se puede utilizar.
# Los bloques que no se pueden utilizar PARA FILTRAR sí se pueden utilizar para luego inferir información de probabilidad de nacionalidad, residencia, ...
# El bloque 5 no lo hemos utilizdo ya que de momento no he encontrado información y tampoco sé si podremos.